In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Using Solver Options

All solvers (both nonlinear and linear) have a number of options that you access via the `options` attribute that control its behavior:

In [2]:
import openmdao.api as om
om.show_options_table("openmdao.solvers.solver.Solver")

Option,Default,Acceptable Values,Acceptable Types,Description
atol,1e-10,N/A,N/A,absolute error tolerance
err_on_non_converge,False,"[True, False]",['bool'],"When True, AnalysisError will be raised if we don't converge."
iprint,1,N/A,['int'],whether to print output
maxiter,10,N/A,['int'],maximum number of iterations
rtol,1e-10,N/A,N/A,relative error tolerance


## Iteration Limit and Convergence Tolerances

Here is how you would change the iteration limit and convergence tolerances for [NonlinearBlockGS](../../building_blocks/solvers/nonlinear_block_gs).

In [3]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src46", get_code("openmdao.test_suite.components.sellar.SellarDis1withDerivatives"), display=False)

:::{Admonition} `SellarDis1withDerivatives` class definition 
:class: dropdown

{glue:}`code_src46`
:::

In [4]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src47", get_code("openmdao.test_suite.components.sellar.SellarDis2withDerivatives"), display=False)

:::{Admonition} `SellarDis2withDerivatives` class definition 
:class: dropdown

{glue:}`code_src47`
:::

In [5]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src48", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src48`
:::

In [6]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())

nlbgs = prob.model.nonlinear_solver = om.NonlinearBlockGS()

nlbgs.options['maxiter'] = 20
nlbgs.options['atol'] = 1e-6
nlbgs.options['rtol'] = 1e-6

prob.setup()
prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))
prob.run_model()

print(prob.get_val('y1'))
print(prob.get_val('y2'))

NL: NLBGS Converged in 5 iterations
[25.5883027]
[12.05848818]


In [7]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('y1'), 25.58830273, .00001)
assert_near_equal(prob.get_val('y2'), 12.05848819, .00001)

5.538631102517653e-10

## Displaying Solver Convergence Info

Solvers can all print out some information about their convergence history. If you want to control that printing behavior you can use the `iprint` option in the solver.


**iprint = -1**: Print nothing

In [8]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()

prob.setup()

newton = prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
scipy = prob.model.linear_solver = om.ScipyKrylov()

newton.options['maxiter'] = 2

# use a real bad initial guess
prob.set_val('y1', 10000)
prob.set_val('y2', -26)

newton.options['iprint'] = -1
scipy.options['iprint'] = -1
prob.run_model()


**iprint = 0**: Print only errors or convergence failures.

In [9]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()

prob.setup()

newton = prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
scipy = prob.model.linear_solver = om.ScipyKrylov()

newton.options['maxiter'] = 1

prob.set_val('y1', 10000)
prob.set_val('y2', -26)

newton.options['iprint'] = 0
scipy.options['iprint'] = 0

prob.run_model()


NL: NewtonSolver 'NL: Newton' on system '' failed to converge in 1 iterations.



**iprint = 1**: Print a convergence summary, as well as errors and convergence failures.

In [10]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())

prob.setup()

newton = prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
scipy = prob.model.linear_solver = om.ScipyKrylov()

newton.options['maxiter'] = 20

prob.set_val('y1', 10000)
prob.set_val('y2', -26)

newton.options['iprint'] = 1
scipy.options['iprint'] = 0
prob.run_model()

NL: Newton Converged in 3 iterations



**iprint = 2**: Print the residual for every solver iteration.

In [11]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())

prob.setup()

newton = prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
scipy = prob.model.linear_solver = om.ScipyKrylov()

newton.options['maxiter'] = 20

prob.set_val('y1', 10000)
prob.set_val('y2', -26)

newton.options['iprint'] = 2
scipy.options['iprint'] = 1
prob.run_model()

NL: Newton 0 ; 1.95729619e+11 1
NL: Newton 1 ; 1.60660573e+13 82.0829129
NL: Newton 2 ; 1275.58483 6.51707613e-09
NL: Newton 3 ; 4.68709816 2.39468005e-11
NL: Newton Converged


(solver-options-set_solver-print)=
## Controlling Solver Output in Large Models

When you have a large model with multiple solvers, it is easier to use a shortcut method that
recurses over the entire model. The `set_solver_print` method on `problem` can be used to
set the iprint to one of four specific values for all solvers in the model while specifically
controlling depth (how many systems deep) and the solver type (linear, nonlinear, or both.)

To print everything, just call `set_solver_print` with a level of "2".

In [12]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src49", get_code("openmdao.test_suite.components.double_sellar.SubSellar"), display=False)

:::{Admonition} `SubSellar` class definition 
:class: dropdown

{glue:}`code_src49`
:::

In [13]:
import numpy as np
from openmdao.test_suite.components.double_sellar import SubSellar

prob = om.Problem()
model = prob.model

sub1 = model.add_subsystem('sub1', om.Group(), promotes_inputs=['z'])
sub2 = sub1.add_subsystem('sub2', om.Group(), promotes_inputs=['z'])
g1 = sub2.add_subsystem('g1', SubSellar(), promotes_inputs=['z'])
g2 = model.add_subsystem('g2', SubSellar())

model.connect('sub1.sub2.g1.y2', 'g2.x')
model.connect('g2.y2', 'sub1.sub2.g1.x')

model.nonlinear_solver = om.NewtonSolver()
model.linear_solver = om.ScipyKrylov()
model.nonlinear_solver.options['solve_subsystems'] = True
model.nonlinear_solver.options['max_sub_solves'] = 0

g1.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g1.linear_solver = om.LinearBlockGS()

g2.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g2.linear_solver = om.ScipyKrylov()
g2.linear_solver.precon = om.LinearBlockGS()
g2.linear_solver.precon.options['maxiter'] = 2

prob.set_solver_print(level=2)

prob.setup()
prob.set_val('z', np.array([5.0, 2.0]))
prob.run_model()

+  
+  ============
+  sub1.sub2.g1
+  ============
+  NL: Newton 0 ; 27.6990975 1
+  |  
+  |  ============
+  |  sub1.sub2.g1
+  |  ============
+  |  LN: LNBGS 0 ; 27.6990975 1
+  |  LN: LNBGS 1 ; 4.08 0.147297218
+  |  LN: LNBGS 2 ; 0.408 0.0147297218
+  |  LN: LNBGS 3 ; 0.0408 0.00147297218
+  |  LN: LNBGS 4 ; 0.00408 0.000147297218
+  |  LN: LNBGS 5 ; 0.000408 1.47297218e-05
+  |  LN: LNBGS 6 ; 4.08e-05 1.47297218e-06
+  |  LN: LNBGS 7 ; 4.08e-06 1.47297218e-07
+  |  LN: LNBGS 8 ; 4.07999998e-07 1.47297217e-08
+  |  LN: LNBGS 9 ; 4.07999998e-08 1.47297217e-09
+  |  LN: LNBGS 10 ; 4.08000034e-09 1.4729723e-10
+  |  LN: LNBGSSolver 'LN: LNBGS' on system 'sub1.sub2.g1' failed to converge in 10 iterations.
+  |  LS: BCHK 0 ; 7.63720546 0.275720372
+  NL: Newton 1 ; 7.63720546 0.275720372
+  |  
+  |  ============
+  |  sub1.sub2.g1
+  |  ============
+  |  LN: LNBGS 0 ; 35.8626295 1
+  |  LN: LNBGS 1 ; 5.31210051 0.148123565
+  |  LN: LNBGS 2 ; 0.108228014 0.00301784938
+  |  LN: LNB

To print everything for nonlinear solvers, and nothing for the linear solvers, first turn everything
on, as shown above, and then call `set_solver_print` again to set a level of "-1" on just the linear solvers (using the `type_` argument), so that we suppress everything, including the messages when the linear block Gauss-Seidel solver hits the maximum iteration limit. You can call the `set_solver_print` method multiple times to stack different solver print types in your model.

In [14]:
from openmdao.test_suite.components.double_sellar import SubSellar

prob = om.Problem()
model = prob.model

sub1 = model.add_subsystem('sub1', om.Group(), promotes_inputs=['z'])
sub2 = sub1.add_subsystem('sub2', om.Group(), promotes_inputs=['z'])
g1 = sub2.add_subsystem('g1', SubSellar(), promotes_inputs=['z'])
g2 = model.add_subsystem('g2', SubSellar())

model.connect('sub1.sub2.g1.y2', 'g2.x')
model.connect('g2.y2', 'sub1.sub2.g1.x')

model.nonlinear_solver = om.NewtonSolver()
model.linear_solver = om.ScipyKrylov()
model.nonlinear_solver.options['solve_subsystems'] = True
model.nonlinear_solver.options['max_sub_solves'] = 0

g1.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g1.linear_solver = om.LinearBlockGS()

g2.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g2.linear_solver = om.ScipyKrylov()
g2.linear_solver.precon = om.LinearBlockGS()
g2.linear_solver.precon.options['maxiter'] = 2

prob.set_solver_print(level=2)
prob.set_solver_print(level=-1, type_='LN')

prob.setup()
prob.set_val('z', np.array([5.0, 2.0]))
prob.run_model()

+  
+  ============
+  sub1.sub2.g1
+  ============
+  NL: Newton 0 ; 27.6990975 1
+  |  LS: BCHK 0 ; 7.63720546 0.275720372
+  NL: Newton 1 ; 7.63720546 0.275720372
+  |  LS: BCHK 0 ; 0.00229800815 0.000300896468
+  NL: Newton 2 ; 0.00229800815 8.29632863e-05
+  |  LS: BCHK 0 ; 2.16279761e-10 9.41161853e-08
+  NL: Newton 3 ; 2.16279761e-10 7.80818802e-12
+  NL: Newton Converged
+  
+  ==
+  g2
+  ==
+  NL: Newton 0 ; 10.8584882 1
+  |  LS: BCHK 0 ; 2.67512104 0.246362201
+  NL: Newton 1 ; 2.67512104 0.246362201
+  |  LS: BCHK 0 ; 0.0161054606 0.0060204605
+  NL: Newton 2 ; 0.0161054606 0.0014832139
+  |  LS: BCHK 0 ; 0.0319283088 1.98245239
+  NL: Newton 3 ; 0.0319283088 0.00294040094
+  |  LS: BCHK 0 ; 6.24223933e-05 0.00195507985
+  NL: Newton 4 ; 6.24223933e-05 5.74871864e-06
+  |  LS: BCHK 0 ; 5.60143795e-05 0.897344311
+  NL: Newton 5 ; 5.60143795e-05 5.15857997e-06
+  |  LS: BCHK 0 ; 2.20095107e-07 0.00392926082
+  NL: Newton 6 ; 2.20095107e-07 2.02694062e-08
+  |  LS: BCHK 0 ; 

If we just want to print solver output for the first level of this multi-level model, we first turn
off all printing, and then set a print level of "2" with a `depth` argument of "2" so that we only print the
top solver and the solver in 'g2', but not the solver in 'sub1.sub2.g1'.

In [15]:
from openmdao.test_suite.components.double_sellar import SubSellar

prob = om.Problem()
model = prob.model

sub1 = model.add_subsystem('sub1', om.Group(), promotes_inputs=['z'])
sub2 = sub1.add_subsystem('sub2', om.Group(), promotes_inputs=['z'])
g1 = sub2.add_subsystem('g1', SubSellar(), promotes_inputs=['z'])
g2 = model.add_subsystem('g2', SubSellar())

model.connect('sub1.sub2.g1.y2', 'g2.x')
model.connect('g2.y2', 'sub1.sub2.g1.x')

model.nonlinear_solver = om.NewtonSolver()
model.linear_solver = om.ScipyKrylov()
model.nonlinear_solver.options['solve_subsystems'] = True
model.nonlinear_solver.options['max_sub_solves'] = 0

g1.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g1.linear_solver = om.LinearBlockGS()

g2.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g2.linear_solver = om.ScipyKrylov()
g2.linear_solver.precon = om.LinearBlockGS()
g2.linear_solver.precon.options['maxiter'] = 2

prob.set_solver_print(level=0)
prob.set_solver_print(level=2, depth=2)

prob.setup()
prob.set_val('z', np.array([5.0, 2.0]))
prob.run_model()

+  |  LN: LNBGSSolver 'LN: LNBGS' on system 'sub1.sub2.g1' failed to converge in 10 iterations.
+  
+  ==
+  g2
+  ==
+  NL: Newton 0 ; 10.8584882 1
+  |  | precon:
+  |  | precon:==
+  |  | precon:g2
+  |  | precon:==
+  |  | precon: LNBGS 0 ; 10.8584882 1
+  |  | precon: LNBGS 1 ; 1.08584882 0.1
+  |  | precon: LNBGS 2 ; 0.108584882 0.01
+  |  | precon:LN: LNBGSSolver 'LN: LNBGS' on system 'g2' failed to converge in 2 iterations.
+  |  | precon:
+  |  | precon:==
+  |  | precon:g2
+  |  | precon:==
+  |  | precon: LNBGS 0 ; 0.108584882 1
+  |  | precon: LNBGS 1 ; 0.0108584882 0.1
+  |  | precon: LNBGS 2 ; 0.00108584882 0.01
+  |  | precon:LN: LNBGSSolver 'LN: LNBGS' on system 'g2' failed to converge in 2 iterations.
+  |  | precon:
+  |  | precon:==
+  |  | precon:g2
+  |  | precon:==
+  |  | precon: LNBGS 0 ; 0 0
+  |  | precon:LN: LNBGS Converged
+  |  LN: SCIPY 0 ; 3.78447274e-16 1
+  |  | precon:
+  |  | precon:==
+  |  | precon:g2
+  |  | precon:==
+  |  | precon: LNBGS 0 ; 10.8

The `set_solver_print` method can also be called on Systems. For instance, if we want to print detailed output from group 'g2' down, we can first call `set_solver_print` on the problem or the top level model with a level of "-1", and then call it on group 'g2' with a level of "2".

In [16]:
from openmdao.test_suite.components.double_sellar import SubSellar

prob = om.Problem()
model = prob.model

model.add_subsystem('pz', om.IndepVarComp('z', np.array([5.0, 2.0])))

sub1 = model.add_subsystem('sub1', om.Group())
sub2 = sub1.add_subsystem('sub2', om.Group())
g1 = sub2.add_subsystem('g1', SubSellar())
g2 = model.add_subsystem('g2', SubSellar())

model.connect('sub1.sub2.g1.y2', 'g2.x')
model.connect('g2.y2', 'sub1.sub2.g1.x')

model.nonlinear_solver = om.NewtonSolver()
model.linear_solver = om.ScipyKrylov()
model.nonlinear_solver.options['solve_subsystems'] = True
model.nonlinear_solver.options['max_sub_solves'] = 0

g1.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g1.linear_solver = om.LinearBlockGS()

g2.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g2.linear_solver = om.ScipyKrylov()
g2.linear_solver.precon = om.LinearBlockGS()
g2.linear_solver.precon.options['maxiter'] = 2

prob.set_solver_print(level=-1, type_='all')
g2.set_solver_print(level=2, type_='NL')

prob.setup()
prob.run_model()

+  
+  ==
+  g2
+  ==
+  NL: Newton 0 ; 0.295012438 1
+  |  LS: BCHK 0 ; 0.0104440989 0.0354022325
+  NL: Newton 1 ; 0.0104440989 0.0354022325
+  |  LS: BCHK 0 ; 0.00675771797 0.647036955
+  NL: Newton 2 ; 0.00675771797 0.0229065527
+  |  LS: BCHK 0 ; 7.20504587e-05 0.0106619511
+  NL: Newton 3 ; 7.20504587e-05 0.000244228546
+  |  LS: BCHK 0 ; 1.04347169e-06 0.0144825128
+  NL: Newton 4 ; 1.04347169e-06 3.53704303e-06
+  |  LS: BCHK 0 ; 2.34603612e-06 2.24829878
+  NL: Newton 5 ; 2.34603612e-06 7.95232953e-06
+  |  LS: BCHK 0 ; 3.01358217e-08 0.012845421
+  NL: Newton 6 ; 3.01358217e-08 1.02151021e-07
+  |  LS: BCHK 0 ; 1.36617051e-09 0.0453337733
+  NL: Newton 7 ; 1.36617051e-09 4.63089123e-09
+  |  LS: BCHK 0 ; 7.65679853e-10 0.56045702
+  NL: Newton 8 ; 7.65679853e-10 2.5954155e-09
+  |  LS: BCHK 0 ; 7.33850758e-11 0.0958430283
+  NL: Newton 9 ; 7.33850758e-11 2.48752481e-10
+  NL: Newton Converged
